<a href="https://colab.research.google.com/github/nntadotzip/XLNet-sandbox/blob/main/XLNetQA_huggingfacePipeline_bertTokenizer_dataVer12April2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Install

In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
! pip install datasets transformers
! pip install pytorch-transformers

In [6]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machin

In [7]:
import torch
import transformers
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import XLNetModel, XLNetTokenizer, XLNetForQuestionAnswering
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [9]:
!git config --global credential.helper store

In [10]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [11]:
!apt install git-lfs
!git config --global user.email "nntanh@hcmiu.edu.vn"
!git config --global user.name "nntadotzip"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,738 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [12]:
from datasets import load_dataset
dataset = load_dataset("nntadotzip/iuQAchatbot_ver12April2022", use_auth_token=True, field='data')

Using custom data configuration nntadotzip--iuQAchatbot_ver12April2022-546da607794b632c


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/nntadotzip--iuQAchatbot_ver12April2022-546da607794b632c/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 630
    })
    test: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 70
    })
})

In [14]:
dataset["train"][0]

{'answers': {'answer_start': [1443],
  'text': ['Ngành Kỹ thuật Hệ thống Công nghiệp (Mã ngành: 7520118) Chương trình: Trong nước Chương trình đại học của ngành Kỹ thuật Hệ thống công nghiệp (ISE) sẽ cung cấp cho sinh viên những kiến thức lý thuyết và kỹ năng thực tế trong việc thiết kế, vận hành, cải thiện và tối ưu các hệ thống sản xuất và dịch vụ Sau khi tốt nghiệp, sinh viên có thể:, Theo đuổi việc học lên cao hơn, có khả năng cạnh tranh thành công để giành được những học bổng uy tín để có thể theo học các chương trình cao học ở nước ngoài Trở thành kỹ sư Kỹ thuật Hệ thống công nghiệp giỏi, có thể tìm được việc làm trong các lĩnh vực sản xuất, quản lý vật tư tồn kho, chuỗi cung ứng và hậu cần, quản lý chất lượng, lập kế hoạch và điều độ, thiết kế và sắp xếp các mặt bằng trong các công ty/ cơquan: -']},
 'context': 'Ngành Công nghệ Thông tin (Mã ngành: 7480201) Chương trình: Trong nước Công nghệ Thông tin (với hai chuyên ngành: kỹ sư mạng máy tính và cử nhân khoa học máy tính) và cử

Review the Dataset

In [15]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(dataset["train"])

,answers,context,id,question,title
0,"{'answer_start': [0], 'text': ['Phương pháp nghiên cứu trong tài chính(MAFE403IU) Research Methods in Finance - Quản trị rủi ro tài chính 2(MAFE404IU) Financial Risk Management 2 - Toán Tài Chính 2(MAFE401IU) Financial Mathematics 2 - Quản Trị Danh Mục Đầu Tư(MAFE402IU) Portfolio Management - Quá trình ngẫu nhiên(MAFE302IU) Random Processes - Tối ưu hóa 1(MAFE303IU) Optimization 1 - Giải tích thực(MAFE201IU) Real Analysis - Phương trình vi phân(MAFE202IU) Differential Equations - Mạng máy tính(MAFE205IU) Computer Networks - Giải tích 3(MAFE203IU) Analysis 3 - Hệ quản trị cơ sở dữ liệu(MAFE204IU) Database Management system - Thị trường tài chính(MAFE209IU) Financial markets - Kỹ thuật ra quyết định(MAFE207IU) Decision Making - Giải tích số(MAFE208IU) Numerical Analysis - Đại số tuyến tính(MAFE104IU) Linear Algebra - Kinh tế tài chính(MAFE105IU) Financial Economics - Khai phá dữ liệu(MAFE312IU) Data mining - Định giá Tài sản(MAFE311IU) Asset pricing - Kinh tế lượng trong tài chính(MAFE314IU) Financial Econometrics - Giải tích 1(MAFE101IU) Analysis 1 - Lập trình C/C++(MAFE102IU) Programming C/C++ - Nhập môn Tài chính doanh nghiệp(MAFE315IU) Introduction to corporate finance - Toán Tài Chính 1(MAFE306IU) Financial Mathematics 1 - Tối ưu hóa 2(MAFE307IU) Optimization 2 - Quản trị rủi ro tài chính 1(MAFE308IU) Financial Risk Management 1 - Kỹ thuật phần mềm(MAFE309IU) Software Engineering - Kinh tế vi mô(BA117IU) Introduction to Micro Economics - Luận văn tốt nghiệp(MAFE409IU) Graduation Thesis - Kinh tế vĩ mô(BA119IU) Introduction to Macro Economics - Quản trị tài chính(MAFE215IU) Financial Management -']}",Phương pháp nghiên cứu trong tài chính(MAFE403IU) Research Methods in Finance - Quản trị rủi ro tài chính 2(MAFE404IU) Financial Risk Management 2 - Toán Tài Chính 2(MAFE401IU) Financial Mathematics 2 - Quản Trị Danh Mục Đầu Tư(MAFE402IU) Portfolio Management - Quá trình ngẫu nhiên(MAFE302IU) Random Processes - Tối ưu hóa 1(MAFE303IU) Optimization 1 - Giải tích thực(MAFE201IU) Real Analysis - Phương trình vi phân(MAFE202IU) Differential Equations - Mạng máy tính(MAFE205IU) Computer Networks - Giải tích 3(MAFE203IU) Analysis 3 - Hệ quản trị cơ sở dữ liệu(MAFE204IU) Database Management system - Thị trường tài chính(MAFE209IU) Financial markets - Kỹ thuật ra quyết định(MAFE207IU) Decision Making - Giải tích số(MAFE208IU) Numerical Analysis - Đại số tuyến tính(MAFE104IU) Linear Algebra - Kinh tế tài chính(MAFE105IU) Financial Economics - Khai phá dữ liệu(MAFE312IU) Data mining - Định giá Tài sản(MAFE311IU) Asset pricing - Kinh tế lượng trong tài chính(MAFE314IU) Financial Econometrics - Giải tích 1(MAFE101IU) Analysis 1 - Lập trình C/C++(MAFE102IU) Programming C/C++ - Nhập môn Tài chính doanh nghiệp(MAFE315IU) Introduction to corporate finance - Toán Tài Chính 1(MAFE306IU) Financial Mathematics 1 - Tối ưu hóa 2(MAFE307IU) Optimization 2 - Quản trị rủi ro tài chính 1(MAFE308IU) Financial Risk Management 1 - Kỹ thuật phần mềm(MAFE309IU) Software Engineering - Kinh tế vi mô(BA117IU) Introduction to Micro Economics - Luận văn tốt nghiệp(MAFE409IU) Graduation Thesis - Kinh tế vĩ mô(BA119IU) Introduction to Macro Economics - Quản trị tài chính(MAFE215IU) Financial Management -,176,Cho tôi biết chương trình ngành Toán ứng dụng (Kỹ thuật Tài chính và Quản trị rủi ro) ?,Chương trình
1,"{'answer_start': [1594], 'text': ['Ngành Kỹ thuật Hệ thống Công nghiệp (Mã ngành: 7520118) Chương trình: Trong nước Chương trình đại học của ngành Kỹ thuật Hệ thống công nghiệp (ISE) sẽ cung cấp cho sinh viên những kiến thức lý thuyết và kỹ năng thực tế trong việc thiết kế, vận hành, cải thiện và tối ưu các hệ thống sản xuất và dịch vụ Sau khi tốt nghiệp, sinh viên có thể:, Theo đuổi việc học lên cao hơn, có khả năng cạnh tranh thành công để giành được những học bổng uy tín để có thể theo học các chương trình cao học ở nước ngoài Trở thành kỹ sư Kỹ thuật Hệ thống công nghiệp giỏi, có thể tìm được việc làm trong các lĩnh vự

# Preprocess the training data

In [17]:
model_checkpoint = "xlnet-base-cased"
batch_size = 8

In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [20]:
# import transformers
# assert isinstance(tokenizer, transformers.XLNetTokenizerFast)

In [21]:
tokenizer("Phòng Bảo đảm Chất lượng Giáo dục và Khảo thí:", " Tư vấn, tham mưu cho Ban Giám hiệu về chiến lược, các kế hoạch cải tiến hoạt động bảo đảm chất lượng")

{'input_ids': [101, 7642, 20142, 2118, 139, 28641, 1186, 288, 28641, 1306, 20394, 28642, 1204, 181, 28265, 28660, 2118, 144, 1182, 5589, 1186, 173, 28661, 1665, 191, 9183, 148, 1324, 28641, 1186, 24438, 6212, 131, 102, 157, 28265, 191, 28642, 1179, 117, 24438, 2312, 182, 28265, 1358, 22572, 1186, 18393, 144, 1182, 5589, 1306, 20844, 28651, 1358, 191, 28648, 22572, 1182, 28647, 1179, 181, 28265, 28660, 1665, 117, 172, 5589, 1665, 180, 28647, 16358, 28640, 1732, 172, 28641, 1182, 189, 1182, 28647, 1179, 16358, 28640, 1204, 288, 28657, 2118, 171, 28641, 1186, 288, 28641, 1306, 22572, 28642, 1204, 181, 28265, 28660, 2118, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1,

In [22]:
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [24]:
for i, example in enumerate(dataset["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 512:
        break
example = dataset["train"][i]

Without any truncation, we get the following length for the input IDs:

In [25]:
len(tokenizer(example["question"], example["context"])["input_ids"])

1910

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [26]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

512

Note that we never want to truncate the question, only the context, else the only_second truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with return_overflowing_tokens=True and by passing the stride:

In [27]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of input_ids, but several:

In [28]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] Tôi muốn học môn Kỹ thuật hệ thống thì chọn ngành nào? [SEP] Ngành Công nghệ Thông tin ( Mã ngành : 7480201 ) Chương trình : Trong nước Công nghệ Thông tin ( với hai chuyên ngành : kỹ sư mạng máy tính và cử nhân khoa học máy tính ) và cử nhân Khoa học Dữ liệu. Chương trình đào tạo tuân thủ theo các chuẩn mực quốc tế và đã đạt được chứng nhận kiểm định chất lượng [UNK] Tổ chức các trường Đại học Đông Nam Á ( AUN ) từ năm 2009 và tái kiểm định năm 2017. Các chương trình đào tạo đã đạt được chuẩn MOET năm 2017 và hiện đang tiếp tục thực hiện quy chuẩn kiểm định ABET của Mỹ. - Ngành Kế toán ( Mã ngành : 7340301 ) Chương trình : Trong nước Kế toán ( tiếng Anh là Accountant ) là công việc ghi chép, thu nhận, xử lý và cung cấp các thông tin về tình hình hoạt động tài chính của một tổ chức, một doanh nghiệp, một cơ quan nhà nước, một c [SEP]
[CLS] Tôi muốn học môn Kỹ thuật hệ thống thì chọn ngành nào? [SEP]nh : Trong nước Kế toán ( tiếng Anh là Accountant ) là công việc ghi chép, thu nhậ

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an offset_mapping:

In [29]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 1), (1, 2), (2, 3), (4, 5), (5, 6), (6, 7), (7, 8), (9, 10), (10, 11), (11, 12), (13, 14), (14, 15), (15, 16), (17, 18), (18, 19), (20, 22), (22, 23), (23, 24), (24, 25), (26, 27), (27, 28), (29, 31), (31, 32), (32, 34), (35, 37), (37, 38), (39, 41), (41, 42), (42, 43), (44, 46), (46, 47), (47, 49), (50, 51), (51, 52), (52, 53), (53, 54), (0, 0), (0, 2), (2, 3), (3, 5), (6, 7), (7, 8), (8, 10), (11, 13), (13, 14), (14, 15), (16, 17), (17, 18), (18, 19), (19, 21), (22, 25), (26, 27), (27, 28), (28, 29), (30, 32), (32, 33), (33, 35), (35, 36), (37, 39), (39, 41), (41, 43), (43, 44), (44, 45), (46, 48), (48, 49), (49, 50), (50, 52), (53, 54), (54, 55), (55, 56), (56, 58), (58, 59), (60, 61), (61, 65), (66, 67), (67, 68), (68, 69), (69, 70), (71, 72), (72, 73), (73, 75), (76, 78), (78, 79), (79, 80), (81, 82), (82, 83), (83, 84), (84, 86), (87, 90), (91, 92), (92, 93), (93, 94), (94, 95), (96, 98), (98, 99), (100, 102), (102, 104), (104, 105), (105, 106)]


This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token ([CLS]) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [30]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

T T


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the sequence_ids method of our tokenized_example can be useful:

In [31]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns None for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [32]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

The answer is not in this feature.


And we can double check that it is indeed the theoretical answer:

In [33]:
# print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
# print(answers["text"][0])

In [34]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag allow_impossible_answers is False. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [35]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [36]:
features = prepare_train_features(dataset['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [38]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [39]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2849
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 306
    })
})

# Fine-tune the model

In [40]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForQuestionAnsweringSimple: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-IUChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022",
)

/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:966: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022).
  FutureWarning,


In [42]:
from transformers import default_data_collator

data_collator = default_data_collator

In [43]:
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

In [44]:
# !sudo apt-get install git-lfs

In [45]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022 into local empty directory.


In [46]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2849
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1071


Epoch,Training Loss,Validation Loss
1,No log,0.645125
2,0.841600,0.442829
3,0.522700,0.423975


***** Running Evaluation *****
  Num examples = 306
  Batch size = 8
Saving model checkpoint to xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/checkpoint-500
Configuration saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/checkpoint-500/config.json
Model weights saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/checkpoint-500/pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/checkpoint-500/tokenizer_config.json
Special tokens file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/checkpoint-500/special_tokens_map.json
tokenizer config file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/tokenizer_config.json
Special tokens file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/speci

TrainOutput(global_step=1071, training_loss=0.6686687914708527, metrics={'train_runtime': 3423.7597, 'train_samples_per_second': 2.496, 'train_steps_per_second': 0.313, 'total_flos': 2419364938991616.0, 'train_loss': 0.6686687914708527, 'epoch': 3.0})

In [47]:
trainer.save_model("iuchatbot-trained")

Saving model checkpoint to iuchatbot-trained
Configuration saved in iuchatbot-trained/config.json
Model weights saved in iuchatbot-trained/pytorch_model.bin
tokenizer config file saved in iuchatbot-trained/tokenizer_config.json
Special tokens file saved in iuchatbot-trained/special_tokens_map.json
Saving model checkpoint to xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022
Configuration saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/config.json
Model weights saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/tokenizer_config.json
Special tokens file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/445M [00:00<?, ?B/s]

Upload file runs/Apr14_16-11-26_ab31e9cf965e/events.out.tfevents.1649952695.ab31e9cf965e.71.0:  63%|######3   …

To https://huggingface.co/nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022
   73d7201..7df0c0f  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022
   7df0c0f..3878f66  main -> main



# Evaluation

In [48]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits', 'mems'])

In [49]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([8, 512]), torch.Size([8, 512]))

In [50]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'))

In [51]:
n_best_size = 20

In [52]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [53]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [54]:
validation_features = dataset["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["test"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [55]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `XLNetForQuestionAnsweringSimple.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `XLNetForQuestionAnsweringSimple.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 306
  Batch size = 8


In [56]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [57]:
max_answer_length = 512

In [58]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 11.634724,
  'text': 'Ngành Công nghệ Thông tin (Mã ngành: 7480201) Chương trình: Trong nước Công nghệ Thông tin (với hai chuyên ngành: kỹ sư mạng máy tính và cử nhân khoa học máy tính) và cử nhân Khoa học Dữ liệu. Chương trình đào tạo tuân thủ theo các chuẩn mực quốc tế và đã đạt được chứng nhận kiểm định chất lượng bởi Tổ chức các trường Đại học Đông Nam Á (AUN) từ năm 2009 và tái kiểm định năm 2017. Các chương trình đào tạo đã đạt được chuẩn MOET năm 2017 và hiện đang tiếp tục thực hiện quy chuẩn kiểm định ABET của Mỹ. -'},
 {'score': 7.1981273, 'text': 'Ng'},
 {'score': 6.6377807, 'text': '-'},
 {'score': 6.3602204,
  'text': 'Ngành Công nghệ Thông tin (Mã ngành: 7480201) Chương trình: Trong nước Công nghệ Thông tin (với hai chuyên ngành: kỹ sư mạng máy tính và cử nhân khoa học máy tính) và cử nhân Khoa học Dữ liệu. Chương trình đào tạo tuân thủ theo các chuẩn mực quốc tế và đã đạt được chứng nhận kiểm định chất lượng bởi Tổ chức các trường Đại học Đông Nam Á (AUN) từ nă

In [59]:
dataset["test"][0]["answers"]

{'answer_start': [507],
 'text': ['Ngành Kỹ thuật Xây dựng (Mã ngành: 7580201) Chương trình: Trong nước Kỹ thuật xây dựng là ngành chuyên về lĩnh vực tư vấn, thiết kế, tổ chức thi công, quản lí giảm sát và nghiệm thu các công trình xây dựng phục vụ đời sống con người như: nhà ở, nhà hàng – khách sạn, bệnh viện, cầu đường… -']}

In [60]:
import collections

examples = dataset["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [61]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
            # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [62]:
final_predictions = postprocess_qa_predictions(dataset["test"], validation_features, raw_predictions.predictions)

Post-processing 70 example predictions split into 306 features.


  0%|          | 0/70 [00:00<?, ?it/s]

In [63]:
from datasets import load_metric
metric = load_metric("squad_v2")

In [64]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 0.0,
 'HasAns_f1': 0.0,
 'HasAns_total': 70,
 'best_exact': 0.0,
 'best_exact_thresh': 0.0,
 'best_f1': 0.0,
 'best_f1_thresh': 0.0,
 'exact': 0.0,
 'f1': 0.0,
 'total': 70}

In [65]:
trainer.push_to_hub()

Saving model checkpoint to xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022
Configuration saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/config.json
Model weights saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/tokenizer_config.json
Special tokens file saved in xlnet-base-cased-iuChatbot-ontologyDts-xlnetBaseCased-bertTokenizer-12April2022/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
